In [6]:
!pip install pyngrok

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [2]:
# Importing the drive module from Google Colab to mount Google Drive (Authorization).
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from flask import Flask, request, render_template, jsonify  # Import Flask and other modules for web application
from flask import Flask, send_file  # Import Flask and send_file for serving images
from io import BytesIO  # Import BytesIO for handling binary data
import matplotlib.pyplot as plt  # Import matplotlib for plotting images
from PIL import Image  # Import PIL for image processing
import matplotlib.image as mpimg  # Import matplotlib.image for reading images
from ultralytics import YOLO # YOLO object detection model
import mediapipe as mp # Mediapipe for pose estimation
import cv2 # OpenCV for image processing
import os # For file operations
import random # For random number generation
from matplotlib.patches import Rectangle  # For drawing rectangles on images

import math  # For mathematical operations
import io  # For handling binary data
import base64  # For encoding/decoding binary data
import numpy as np  # For numerical operations
from operator import imod  # For modulo operation

from pyngrok import ngrok  # For exposing local server to the internet


In [11]:
# Load a model
model = YOLO('yolov8n.pt')  # load an official model

model = YOLO("/content/yolov8n.pt")  # load a custom model

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token("2gTWkFi5q5FaBt0VyACLQCPfCwT_3k1vzLFMqsfaHBMXs5zcT")
public_url =  ngrok.connect(5000).public_url
print(public_url)
model = YOLO("/content/drive/MyDrive/TFQD API/best.pt")  # load a custom model

@app.route('/Ruku', methods=['POST', 'GET'])
def Detect_Ruku():
    imagefile = request.files['imagefile']
    image_path = imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()


    # Function to calculate angle between lines formed by three points
    def calculate_angle(a, b, c):
        angle_rad1 = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
        angle_rad1 = angle_rad1 % (2 * math.pi)
        angle_rad2 = math.atan2(a[1]-b[1], a[0]-b[0]) - math.atan2(c[1]-b[1], c[0]-b[0])
        angle_rad2 = angle_rad2 % (2 * math.pi)
        return math.degrees(min(angle_rad1, angle_rad2))

    # Function to check if the pose resembles the ruku position in Salat
    def is_ruku_pose(landmarks):
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
        right_hip = (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
        right_shoulder = (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y)
        right_knee = (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)

        angle_left = abs(90 - calculate_angle(left_knee, left_hip, left_shoulder))
        angle_right = abs(90 - calculate_angle(right_knee, right_hip, right_shoulder))

        ruku_angle_threshold = 30  # Adjust as needed
        if angle_left <= ruku_angle_threshold or angle_right <= ruku_angle_threshold:
            return True, min(abs(angle_left), abs(angle_right)) / 100 * 100 # Calculate error percentage
        else:
            return False, min(abs(angle_left), abs(angle_right)) / 100 * 100

    def process_image(image_path):
        results = model.predict(image_path)

        for r in results:
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()

        bounding_box = tuple(int(value) for value in bounding_box)

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        is_ruku, error_percentage = is_ruku_pose(results.pose_landmarks.landmark)


        # Red color if error>15, else green

        if error_percentage<= 15:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3

            # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)


        Perfect_Threshold = 5;


        plt.figure(figsize=(8, 8))
        plt.imshow(image_rgb)
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        for landmark in [mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
                        mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                        mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
            plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                        results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                        color='white', s=100)

        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE.value].y * image.shape[0]],
                color='cyan', linestyle='-', linewidth=2, alpha=0.5)
        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * image.shape[0]],
                color='cyan', linestyle='-', linewidth=2, alpha=0.5)

        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image.shape[0]],
                color='magenta', linestyle='-', linewidth=2, alpha=0.5)
        plt.plot([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * image.shape[1]],
                [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0],
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image.shape[0]],
                color='magenta', linestyle='-', linewidth=3, alpha=0.5)

        print('Done')
        plt.axis('off')
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()
        return image_base64, error_percentage, is_ruku



    # Process the image and get the base64 encoded plot
    plot_base64, error, is_ruku= process_image(image_path)
    error_text = "{:.2f}".format(error)

    if is_ruku and error <= 5:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاة بالصوره المرفوعه وضعية ركوع مثالي"})
    if is_ruku and error>5 and error<=15:
        return jsonify({'image_base64': plot_base64, 'error_description': " ركوع صحيح لكن فيه نسبة انحراف عن الأصل بمقدار: {} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    if is_ruku and error>15 and error<=30:
        return jsonify({'image_base64': plot_base64, 'error_description': " ركوع صحيح لكن فيه نسبة انحراف كبيرة عن الأصل بمقدار:{} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    else:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاه بالصوره المرفوعه ليست وضعية ركوع"})


@app.route('/Takbeer', methods=['POST', 'GET'])
def Detect_Takbeer():
    imagefile = request.files['imagefile']
    image_path =  imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Function to calculate angle between lines formed by three points
    def calculate_angle(a, b, c):
        angle_rad = math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])
        angle_rad = angle_rad % (2 * math.pi)
        return math.degrees(angle_rad)

    # Function to check if the pose resembles the Takbeer position in Salat
    def is_takbeer_pose(landmarks):
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
        right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        print(abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.22 * 100)

        Perfect_Threshold = 5;
        # Check if left and right wrist are on the same line
        if abs(left_wrist.y - right_wrist.y) < 0.05 and abs(left_wrist.x - right_wrist.x) > 0.15:
            # Check if distance between right hip and right shoulder in y-direction is big enough
            if right_hip.y - right_shoulder.y > 0.25 or left_hip.y - left_shoulder.y > 0.25 :
              return True, abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.25 * 100
        return False, abs(abs(left_wrist.y)-abs(right_wrist.y)) / 0.25 * 100

    def process_image(image_path):
        model_results = model.predict(image_path)

        for r in model_results:
            print(r.boxes)
            # Extract bounding box coordinates
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
            print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
            bounding_box_tuple = tuple(bounding_box)

        bounding_box = tuple(int(value) for value in bounding_box_tuple)

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Detect pose landmarks
        results = pose.process(image_rgb)
        is_takbeer, error = is_takbeer_pose(results.pose_landmarks.landmark)
        print(error)

        # Red color if error>15, else green

        if error<= 15 and is_takbeer:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3


        # Load image
        # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)

        # Convert image to RGB

        Perfect_Threshold = 5;



        # Draw text above bounding box with background color
        # Draw text above bounding box with background color
        plt.figure(figsize=(8, 8))
        plt.imshow(image_rgb)

        # Draw connections between all specified landmarks
        landmarks = results.pose_landmarks.landmark
        connections = [(mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                      (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
                      (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
                      (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
                      (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_WRIST),
                      (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_WRIST),
                      (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX),
                      (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_INDEX)]

        # Draw points for knees, hips, and shoulders
        for landmark in [mp_pose.PoseLandmark.LEFT_INDEX, mp_pose.PoseLandmark.RIGHT_INDEX,
                        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
                            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
                plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                            results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                            color='white', s=100)

        for connection in connections:
            start_point = connection[0]
            end_point = connection[1]
            plt.plot([landmarks[start_point.value].x * image.shape[1], landmarks[end_point.value].x * image.shape[1]],
                    [landmarks[start_point.value].y * image.shape[0], landmarks[end_point.value].y * image.shape[0]],
                    color='magenta', linestyle='-', linewidth=1)
        # Draw landmarks and connections
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=5),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))
        plt.axis('off')
        # Save the plot as bytes in memory
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        # Convert the plot bytes to base64 string
        plot_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        plt.close()  # Close the plot to free memory

        return plot_base64, error, is_takbeer

    # Process the image and get the base64 encoded plot
    plot_base64,error, is_takbeer = process_image(image_path)

    error_text = "{:.2f}".format(error)


    if is_takbeer and error <= 5:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاة بالصوره المرفوعه وضعية تكبير مثاليه"})
    if is_takbeer and error>5 and error<=15:
        return jsonify({'image_base64': plot_base64, 'error_description': " تكبير صحيح لكن فيه نسبة انحراف عن الأصل بمقدار:{} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    if is_takbeer and error>15 and error<=30:
        return jsonify({'image_base64': plot_base64, 'error_description': " تكبير صحيح لكن فيه نسبة انحراف كبيرة عن الأصل بمقدار:{} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    else:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاه بالصوره المرفوعه ليست وضعية تكبير"})



@app.route('/Raising', methods=['POST', 'GET'])
def Detect_Raising():
    imagefile = request.files['imagefile']
    image_path = imagefile.filename
    imagefile.save(image_path)

    results = model.predict(image_path)


    for r in results:
        # Extract bounding box coordinates
        bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
        print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
        bounding_box_tuple = tuple(bounding_box)


    bounding_box_int = tuple(int(value) for value in bounding_box_tuple)

    print('The predicted class is ' + str(bounding_box_int))


    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    def is_raising_pose(landmarks):
        # Extract relevant landmarks
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
        nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
        chin = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

        print(abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100)
        print(left_wrist.x - right_wrist.x)
        print(left_wrist.y - right_wrist.y)

        # Check if left and right wrist are close to each other
        if abs(left_wrist.x - right_wrist.x) and abs(left_wrist.y - right_wrist.y) < 0.05:
            # Check if distance between right hip and right shoulder in y-direction is big enough
            if right_hip.y - right_shoulder.y > 0.25 or left_hip.y - left_shoulder.y > 0.25 :
                if right_wrist.y - right_shoulder.y > 0.15 or left_wrist.y - left_shoulder.y > 0.15:
                    # Check  wrists are above the shoulders
                    return True, abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100
        return False, abs(abs(abs(left_wrist.y)-abs(right_wrist.y))+abs(abs(left_wrist.x)-abs(right_wrist.x))-0.03)/ 2.5 * 100

    def process_image(image_path):
        # Load Model
        results = model.predict(image_path)

        for r in results:
            print(r.boxes)
            # Extract bounding box coordinates
            bounding_box = x1, y1, x2, y2 = r.boxes.xyxy[0].tolist()
            print("Bounding box coordinates (x1, y1, x2, y2):", x1, y1, x2, y2)
            bounding_box_tuple = tuple(bounding_box)

        bounding_box = tuple(int(value) for value in bounding_box_tuple)

        # Load image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Detect pose landmarks
        results = pose.process(image_rgb)
        is_raising, error = is_raising_pose(results.pose_landmarks.landmark)




        # Red color if error>15, else green

        if error<= 15 and is_raising:
          color = (0, 255, 0)
        else:
          color = (255, 0, 0)

        # Line thickness of 3 px
        thickness = 3


        # Draw bounding box
        x1, y1, x2, y2 = bounding_box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)

        # Convert image to RGB

        Perfect_Threshold = 5;



        # Check if pose resembles raising pose

        # Draw text above bounding box with background color
        plt.figure(figsize=(8, 8))
        plt.savefig(image_path)  # Specify the file name and extension
        plt.imshow(image_rgb)

        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Draw connections between all specified landmarks
        landmarks = results.pose_landmarks.landmark
        connections = [(mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
                    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
                    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
                    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_WRIST),
                    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_WRIST),
                    (mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX),
                    (mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_INDEX)]

        # Draw points for knees, hips, and shoulders
        for landmark in [mp_pose.PoseLandmark.LEFT_INDEX, mp_pose.PoseLandmark.RIGHT_INDEX,
                        mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
                            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
                            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER]:
                plt.scatter(results.pose_landmarks.landmark[landmark.value].x * image.shape[1],
                            results.pose_landmarks.landmark[landmark.value].y * image.shape[0],
                            color='white', s=100)

        for connection in connections:
            start_point = connection[0]
            end_point = connection[1]
            plt.plot([landmarks[start_point.value].x * image.shape[1], landmarks[end_point.value].x * image.shape[1]],
                    [landmarks[start_point.value].y * image.shape[0], landmarks[end_point.value].y * image.shape[0]],
                    color='magenta', linestyle='-', linewidth=1)
        # Draw landmarks and connections
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing.draw_landmarks(image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=5),
                                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))

        plt.axis('off')
        # Save the plot as bytes in memory
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)
        # Convert the plot bytes to base64 string
        plot_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

        plt.close()  # Close the plot to free memory

        return plot_base64, error, is_raising

    # Process the image and get the base64 encoded plot
    plot_base64, error, is_raising = process_image(image_path)
    error_text = "{:.2f}".format(error)


    if is_raising and error <= 5:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاة بالصوره المرفوعه وضعية استقامه مثاليه"})
    if is_raising and error>5 and error<=15:
        return jsonify({'image_base64': plot_base64, 'error_description': " استقامه صحيحه لكن فيه نسبة انحراف عن الأصل بمقدار:{} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    if is_raising and error>15 and error<=30:
        return jsonify({'image_base64': plot_base64, 'error_description': " استقامه صحيح لكن فيه نسبة انحراف كبيرة عن الأصل بمقدار:{} % للتأكد من الصفة الصحيحة يمكنك مراجعة ارشاداتنا".format(error_text)})
    else:
        return jsonify({'image_base64': plot_base64, 'error_description': "وضعية الصلاه بالصوره المرفوعه ليست وضعية استقامه"})


if __name__ == "__main__":
    app.run(port=5000)


https://242f-35-227-136-47.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



image 1/1 /content/Screenshotsruku-1.jpg: 640x448 1 ruku, 81.8ms
Speed: 12.1ms preprocess, 81.8ms inference, 1243.1ms postprocess per image at shape (1, 3, 640, 448)
Bounding box coordinates (x1, y1, x2, y2): 132.9288330078125 416.2718505859375 725.060302734375 1186.594482421875
The predicted class is (132, 416, 725, 1186)

image 1/1 /content/Screenshotsruku-1.jpg: 640x448 1 ruku, 20.6ms
Speed: 3.7ms preprocess, 20.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 448)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.], device='cuda:0')
conf: tensor([0.7949], device='cuda:0')
data: tensor([[1.3293e+02, 4.1627e+02, 7.2506e+02, 1.1866e+03, 7.9489e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1280, 891)
shape: torch.Size([1, 6])
xywh: tensor([[428.9946, 801.4332, 592.1315, 770.3226]], device='cuda:0')
xywhn: tensor([[0.4815, 0.6261, 0.6646, 0.6018]], device='cuda:0')
xyxy: tensor([[ 132.9288,  416.2719,  725.0603, 1186.594

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 16:46:20] "POST /Raising HTTP/1.1" 200 -



image 1/1 /content/takbeer-1.jpg: 448x640 1 takbeer, 123.6ms
Speed: 9.3ms preprocess, 123.6ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
Bounding box coordinates (x1, y1, x2, y2): 99.73641204833984 0.0 478.6961975097656 301.232666015625
The predicted class is (99, 0, 478, 301)

image 1/1 /content/takbeer-1.jpg: 448x640 1 takbeer, 20.8ms
Speed: 6.8ms preprocess, 20.8ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.], device='cuda:0')
conf: tensor([0.9275], device='cuda:0')
data: tensor([[ 99.7364,   0.0000, 478.6962, 301.2327,   0.9275,   2.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (360, 540)
shape: torch.Size([1, 6])
xywh: tensor([[289.2163, 150.6163, 378.9598, 301.2327]], device='cuda:0')
xywhn: tensor([[0.5356, 0.4184, 0.7018, 0.8368]], device='cuda:0')
xyxy: tensor([[ 99.7364,   0.0000, 478.6962, 301.2327]], device='cuda:0')
xyxyn: tensor([[0.1847, 0

...

INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 17:04:53] "POST /Raising HTTP/1.1" 200 -



image 1/1 /content/raising-1 (3).jpg: 640x640 1 raising, 107.0ms
Speed: 5.4ms preprocess, 107.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Bounding box coordinates (x1, y1, x2, y2): 547.4047241210938 151.12722778320312 2380.06787109375 3023.0
The predicted class is (547, 151, 2380, 3023)

image 1/1 /content/raising-1 (3).jpg: 640x640 1 raising, 29.5ms
Speed: 5.4ms preprocess, 29.5ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9254], device='cuda:0')
data: tensor([[5.4740e+02, 1.5113e+02, 2.3801e+03, 3.0230e+03, 9.2537e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (3023, 2885)
shape: torch.Size([1, 6])
xywh: tensor([[1463.7363, 1587.0636, 1832.6631, 2871.8728]], device='cuda:0')
xywhn: tensor([[0.5074, 0.5250, 0.6352, 0.9500]], device='cuda:0')
xyxy: tensor([[ 547.4047,  151.1272, 2380.0679, 3023.0000]], dev

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


3.6410034179687503
0.11455577611923218
0.006469309329986572


INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 17:06:13] "POST /Raising HTTP/1.1" 200 -


In [ ]:
from google.colab import drive
drive.mount('/content/drive')